In [ ]:
import pandas as pd
import numpy as np

# Set the maximum number of columns to display to 999
pd.options.display.max_columns = 999

# Open the file containing the ribozyme locations
rnarobo=open("../tables_and_result/rnarobo_result_dereplicated.txt","r")

# Read the contents of the file into a list
rnarobol=rnarobo.readlines()

# Close the file
rnarobo.close()

# Create a dictionary to store the start and end positions of each ribozyme
dicti_r={}

# Loop through each line of the ribozyme file, skipping the header and footer
for line in rnarobol[11:-6]:
    # If the line does not start with "|", it contains ribozyme information
    if line[0]!="|":
        # Replace any periods in the line with underscores
        line=line.replace(".","_")
        # Split the line into a list of values
        line=line.strip().split()
        # If the scaffold ID is already in the dictionary, append the start and end positions to the existing list
        if line[2] in dicti_r:
            dicti_r[line[2]].append([int(line[0]),int(line[1])])
        # If the scaffold ID is not in the dictionary, create a new list with the start and end positions
        else:
            dicti_r[line[2]]=[[int(line[0]),int(line[1])]]
        
# Read the coding analysis file into a pandas DataFrame
codings=pd.read_csv("../alternative_coding_res/output/CD_analysis.csv",sep=",")

# Create new DataFrames for each predicted code
codings_15=codings[codings["pred_code"]==15]
codings_4=codings[codings["pred_code"]==4]
codings_11=codings[codings["pred_code"]==11]

# Create a new empty DataFrame to store the ribozyme information
df=pd.DataFrame(columns=["name","start","end","dire","code"])
count=0

# Loop through each scaffold ID in the ribozyme dictionary
for i in dicti_r:
    # Get the scaffold ID
    ids=i
    # Loop through each start and end position for the current scaffold ID
    for x in range(len(dicti_r[ids])):
        # Get the start and end positions
        start=dicti_r[ids][x][0]
        end=dicti_r[ids][x][1]
        # Add the scaffold ID, start position, and end position to the DataFrame
        df.loc[count,"name"]=ids
        df.iloc[count,1]=start
        df.iloc[count,2]=end
        # If the start position is less than the end position, set the direction to "1" (sense)
        if start < end:
            df.iloc[count,3]="1"
        # If the start position is greater than the end position, set the direction to "-1" (antisense)
        else: 
            df.iloc[count,3]="-1"
         
        # Get the predicted code for the current scaffold ID
        try:
            df.iloc[count,4]=codings[codings["Scaffold"]==ids]["pred_code"].values[0]
        # If the predicted code cannot be found, set it to "NA"
        except:
            df.iloc[count,4]="NA"
        count+=1

In [ ]:
# Open the file containing the raw protein sequences for code 11
rawfile=open("../alternative_coding_res/proteins.faa")
rl11=rawfile.readlines()
rawfile.close()

# Open the file containing the raw protein sequences for code 15
rawfile15=open("../alternative_coding_res/proteins15.faa")
rl15=rawfile15.readlines()
rawfile15.close()

# Open the file containing the raw protein sequences for code 4
rawfile4=open("../alternative_coding_res/proteins4.faa")
rl4=rawfile4.readlines()
rawfile4.close()

# Define a function to convert the raw protein sequences to a pandas DataFrame
def raw_to_df_code(code):
    # Initialize empty lists to store the values for each column in the DataFrame
    name=[]
    start=[]
    end=[]
    ids=[]
    dire=[]
    # Create a new empty DataFrame with the appropriate column names
    coding_genes=pd.DataFrame(columns=["name","ids","start","end","dire"])
    # Loop through each line of the raw protein sequence file
    for lin in code:
        # If the line starts with ">", it contains header information
        if lin[0]==">":
            # Get the name of the genome from the header
            nam=lin.strip().split("#")[0][1:]
            name.append(lin.strip().split("#")[0][1:])
            # Get the ID of the coding gene from the header
            ids.append("_".join(nam.split("_")[:-1]))
            # Get the start, end, and direction of the coding gene from the header
            start.append(lin.strip().split("#")[1])
            end.append(lin.strip().split("#")[2])
            dire.append(lin.strip().split("#")[3])

    # Add the values to the DataFrame
    coding_genes["name"]=name
    coding_genes["ids"]=ids
    coding_genes["start"]=start
    coding_genes["end"]=end
    coding_genes["dire"]=dire

    return coding_genes

# Convert the raw protein sequences to DataFrames for each code
coding_genes_15=raw_to_df_code(rl15)
coding_genes_11=raw_to_df_code(rl11)
coding_genes_4=raw_to_df_code(rl4)

# Create a dictionary to store the DataFrames for each code
coding_genes_dic={}
coding_genes_dic["11"]=coding_genes_11
coding_genes_dic["4"]=coding_genes_4
coding_genes_dic["15"]=coding_genes_15



In [25]:
#read is the theta ribozymes

In [ ]:
import pandas as pd
import numpy as np

# Read in the combined tRNA and ribozyme sequence file as a pandas DataFrame
theta=pd.read_csv("../tables_and_result/comb_trna_seq_plus_ribo_fix_march.csv",sep="\t")

# Initialize counters for each category
cat1a=0
cat1b=0
cat2aa=0
cat2ab=0
cat2ba=0
cat2bb=0
cat2c=0
cat3a=0
cat3b=0
cat4a=0
cat4b=0
thet=0

# Initialize an empty list to store the smallest distances between ribozymes and genes
small=[]

# Loop through each row of the DataFrame
for x in range(len(df)):
    # Initialize a variable to store the smallest distance between the ribozyme and a gene
    smallest=10000
    # Get the code, ID, start, end, and direction of the current ribozyme
    code=df.iloc[x,-1]
    name=df.iloc[x,0]
    start=int(df.iloc[x,1])
    end=int(df.iloc[x,2])
    dire=int(df.iloc[x,3])
    # Select the corresponding genes from the called proteins file
    dff=coding_genes_dic[str(code)][coding_genes_dic[str(code)]["ids"]==name]
    # Initialize a variable to check if the ribozyme is a theta ribozyme
    no=0
    
    # Check if the ribozyme is a theta ribozyme
    if name in theta["ID"].values:
        tdf=theta[theta["ID"]==name]
        if start in theta["rnarobo_start"].values:
            thet+=1
            no=2  
            
    # If the ribozyme is not a theta ribozyme
    if no==0:
        for z in range(len(dff)):
            # Get the start, end, and direction of each called gene
            st=int(dff.iloc[z,2])
            en=int(dff.iloc[z,3])
            di=int(dff.iloc[z,4])
            # If the start of the ribozyme is within a gene
            if start in range(st,en+1):
                # If the end of the ribozyme is also within the gene
                if end in range(st,en+1):
                    # Sense
                    if dire==di:
                        cat1a+=1
                        no=1
                    # Antisense
                    else:
                        cat1b+=1
                        no=1
                # If the end of the ribozyme is not within the gene
                else:
                    if dire==di:
                        cat3a+=1
                        no=1
                    else:
                        cat3b+=1
                        no=1
            # If only the end of the ribozyme is in the gene, not the start
            elif end in range(st,en+1):
                # Should not be needed, but keep for consistency in code
                if start in range(st,en+1):
                    # Sense
                    if dire==di:
                        cat1a+=1
                        no=1
                    # Antisense
                    else:
                        cat1b+=1
                        no=1
                # Start is not in the gene
                else:
                    if dire==di:
                        cat3a+=1
                        no=1
                    else:
                        cat3b+=1
                        no=1
            # Check where the ribozyme is closest to the next gene (end or start)
            if abs(start-st) < smallest:
                smallest=abs(start-st)
                if dire==di:
                    dircheck=1
                else:
                    dircheck=0
            if abs(start-en) < smallest:
                smallest=abs(start-en)
                if dire==di:
                    dircheck=1
                else:
                    dircheck=0
            if abs(end-st) < smallest:
                smallest=abs(end-st)
                if dire==di:
                    dircheck=1
                else:
                    dircheck=0
            if abs(end-en) < smallest:
                smallest=abs(end-en)
                if dire==di:
                    dircheck=1
                else:
                    dircheck=0
        # If the ribozyme was neither overlapping nor completely within the gene
        if no==0:
            # Implement length check: how far away is the closest gene (here: <=50 nucleotides away)
            if smallest<=50:
                # Sense
                if dircheck==1:
                    cat2aa+=1
                # Antisense
                else:
                    cat2ab+=1
            elif smallest<=200:
                if dircheck==1:
                    cat2ba+=1
                else:
                    cat2bb+=1
            else:
                if dircheck==1:
                    cat4a+=1
                else:
                    cat4b+=1
            small.append(smallest)


#Print the occurences for each category

print("Cat1a:"+str(cat1a))
print("Cat1b:"+str(cat1b))

print("Cat3a:"+str(cat3a))

print("Cat3b:"+str(cat3b))

print("Cat2aa:"+str(cat2aa))

print("Cat2ab:"+str(cat2ab))

print("Cat2ba:"+str(cat2ba))
print("Cat2bb:"+str(cat2bb))

print("Cat4a:"+str(cat4a))
print("Cat4b:"+str(cat4b))

print("Theta:"+str(thet))


In [99]:
#how many of the theta ribozymes are in range of +-1
t1=theta[abs(theta["rnarobo_start"]-theta["trna_end"]) <2]
t1